# Document level access in Azure AI Search using the indexer pull APIs

In Azure AI Search, you can use an indexer to pull content into a search index for indexing. This notebook shows you how index blobs that have access control lists (ACLs) in Azure Storage Data Lake Storage (ADLS) Gen2, and then query the index to return only those results that the user is authorized to view.

The security principal behind the query access token determines the "user". The ACLs on folders and files determine whether the user is authorized to view the content, and that metadata is pulled into the index along with other document content. Internally at query time, the search engine filters out any documents that aren't associated with the object ID.

This feature is currently in preview.

For an alternative approaching using push APIs to index any data, see [Quickstart-Document-Permissions-Push-API](../Quickstart-Document-Permissions-Push-API/document-permissions-push-api.ipynb).


## Prerequisites

+ Azure AI Search, Basic tier or higher, with a [system-assigned managed identity](https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources) and [role-based access control](https://learn.microsoft.com/azure/search/search-security-enable-roles).

+ Azure Storage, general purpose account, with a [hierarchical namespace](https://learn.microsoft.com/azure/storage/blobs/create-data-lake-storage-account).

+ Folders and files, where each file has an [access control list specified](https://learn.microsoft.com/azure/storage/blobs/data-lake-storage-access-control). We recommend group IDs.

+ [Visual Studio Code](https://code.visualstudio.com/download) with the [Python extension](https://marketplace.visualstudio.com/items?itemName=ms-python.python) and [Jupyter package](https://pypi.org/project/jupyter/).

## Permissions

This walkthrough uses Microsoft Entra ID authentication and authorization.

+ On Azure Storage, **Storage Blob Data Reader** permissions are required for both the search service identity and for your user account since you are testing locally. You also need **Storage Blob Data Contributor** because the sample includes code for creating and configuring a container and its contents.

+ On Azure AI Search, assign yourself **Search Service Contributor**, **Search Index Data Contributor**, and **Search Index Data Reader** permissions to create objects and run queries. For more information, see [Connect to Azure AI Search using roles](https://learn.microsoft.com/azure/search/search-security-rbac) and [Quickstart: Connect without keys for local testing](https://learn.microsoft.com/azure/search/search-get-started-rbac).

## Limitations

+ Parsing indexer options aren't currently supported. There's no support for CSV, JSON, or Markdown parsing.

## Set up connections

Save the `sample.env` file as `.env` and then modify the environment variables to use your Azure endpoints. All variables must be specified.

You need endpoints for:

+ Azure AI Search
+ Azure Storage

For Azure AI Search, find the endpoint in the [Azure portal](https://portal.azure.com), in the **Essentials** section of the Overview page.

For Azure Storage, follow the guidance in [Get storage account configuration information](https://learn.microsoft.com/azure/storage/common/storage-account-get-info) to specify all of the variables in the `.env` file. 

## Load connections

We recommend creating a virtual environment to run this sample code. In Visual Studio Code, open the control palette (ctrl-shift-p) to create an environment. This notebook was tested on Python 3.10.

After the environment is created, load the environment variables to set up connections and object names.

In [ ]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX", "document-permissions-indexer-idx")
indexer_name = os.getenv("AZURE_SEARCH_INDEXER", "document-permissions-indexer-idxr")
datasource_name = os.getenv("AZURE_SEARCH_DATASOURCE", "document-permissions-indexer-ds")
adls_gen2_account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
adls_gen2_container_name = os.getenv("AZURE_STORAGE_CONTAINER_NAME")
adls_gen2_connection_string = os.environ["AZURE_STORAGE_CONNECTION_STRING"]
adls_gen2_resource_id = os.environ["AZURE_STORAGE_RESOURCE_ID"]
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")

## Create an index

The search index must include fields for your content and for permission metadata. Assign the new permission filter option to a string field and make sure the field is filterable. The search engine builds the filter internally at query time.

In [ ]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="content", type="Edm.String", searchable=True, filterable=False, sortable=False),
        SearchField(name="oids", type="Collection(Edm.String)", filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="groups", type="Collection(Edm.String)", filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="metadata_storage_path", type="Edm.String", searchable=True),
        SearchField(name="metadata_storage_name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_or_update_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

## Create a data source

Set the `IndexerPermissionOption` so that the indexer knows to retrieve the permission metadata.

In [ ]:
from azure.search.documents.indexes.models import SearchIndexerDataSourceConnection, SearchIndexerDataSourceType, IndexerPermissionOption, SearchIndexerDataContainer, DataSourceCredentials
from azure.search.documents.indexes import SearchIndexerClient
indexer_client = SearchIndexerClient(endpoint=endpoint, credential=credential)
datasource = SearchIndexerDataSourceConnection(
    name=datasource_name,
    type=SearchIndexerDataSourceType.ADLS_GEN2,
    connection_string=f"ResourceId={adls_gen2_resource_id};",
    container=SearchIndexerDataContainer(name=adls_gen2_container_name),
    indexer_permission_options=[IndexerPermissionOption.GROUP_IDS]
)

indexer_client.create_or_update_data_source_connection(datasource)
print(f"Datasource '{datasource_name}' created with permission filter option enabled.")

## Get group IDs

This step calls the Graph APIs to get a few group IDs for your Microsoft Entra identity. Your group IDs will be added to the access control list of the objects created in the next step. Two group identifiers are retrieved. Each one is assigned to a different file.

In [ ]:
from msgraph import GraphServiceClient
client = GraphServiceClient(credentials=credential, scopes=["https://graph.microsoft.com/.default"])

groups = await client.me.member_of.get()
first_group_id = groups.value[0].id
second_group_id = groups.value[1].id

## Upload sample directory and file

This step creates the container, folders, and uploads the files into Azure Storage. It assigns your group IDs to to the access control list for each file.

In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient
import requests

service = DataLakeServiceClient.from_connection_string(adls_gen2_connection_string, credential=credential)
container = service.get_file_system_client(adls_gen2_container_name)
if not container.exists():
    container.create_file_system()
root_dir_client = container.get_directory_client("/")
state_parks_dir_client = container.get_directory_client("state-parks")
state_parks_dir_client.create_directory()
root_dir_client.update_access_control_recursive(f"group:{first_group_id}:rwx")
root_dir_client.update_access_control_recursive(f"group:{second_group_id}:rwx")

oregon_dir_client = state_parks_dir_client.create_sub_directory("oregon")
oregon_dir_client.create_directory()
file_client = oregon_dir_client.create_file("oregon_state_parks.csv")
oregon_state_parks_content = requests.get("https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/state-parks/Oregon/oregon_state_parks.csv").content.decode("utf-8")
file_client.upload_data(oregon_state_parks_content, overwrite=True)
oregon_dir_client.update_access_control_recursive(f"group:{first_group_id}:rwx")

washington_dir_client = state_parks_dir_client.create_sub_directory("washington")
washington_dir_client.create_directory()
file_client = washington_dir_client.create_file("washington_state_parks.csv")
washington_state_parks_content = requests.get("https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/state-parks/Washington/washington_state_parks.csv").content.decode("utf-8")
file_client.upload_data(washington_state_parks_content, overwrite=True)
washington_dir_client.update_access_control_recursive(f"group:{second_group_id}:rwx")

## Run the indexer

Start the indexer to run all operations, from data retrieval to indexing. Any connection errors or permission problems become evident here.

In [ ]:
from azure.search.documents.indexes.models import SearchIndexer, FieldMapping

indexer = SearchIndexer(
    name=indexer_name,
    target_index_name=index_name,
    data_source_name=datasource_name,
    field_mappings=[
        FieldMapping(source_field_name="metadata_group_ids", target_field_name="groups"),
        FieldMapping(source_field_name="metadata_user_ids", target_field_name="oids"),
    ]
)

indexer_client.create_or_update_indexer(indexer)
print(f"Indexer '{indexer_name}' created")


## Search sample data using x-ms-query-source-authorization

Wait for the indexer to finish processing before running queries. This query uses an empty search string (`*`) for an unqualified search. It returns the file name and permission metadata associated with each file. Notice that each file is associated with a different group ID.

In [ ]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

results = search_client.search(search_text="*", x_ms_query_source_authorization=token_provider(), select="metadata_storage_path,oids,groups", order_by="id asc")
for result in results:
    print(f"Path: {result['metadata_storage_path']}, OID: {result['oids']}, Group: {result['groups']}")

## Search sample data without x-ms-query-source-authorization 

This step demonstrates the user experience when authorization fails. No results are returned in the response.

In [ ]:
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential)

results = search_client.search(search_text="*", x_ms_query_source_authorization=None, select="metadata_storage_path,oids,groups", order_by="id asc")
for result in results:
    print(f"Path: {result['metadata_storage_path']}, OID: {result['oids']}, Group: {result['groups']}")

## Next step

To learn more, see [Document-level access control in Azure AI Search](https://learn.microsoft.com/azure/search/search-document-level-access-overview).